<a href="https://colab.research.google.com/github/Drozeler/Machine-Learning/blob/main/HospitalPatientDiagnosisNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
from spacy.tokens import Span
from spacy.training import Example
from transformers import pipeline

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
labels = ["PATIENT", "AGE", "CHIEF_COMPLAINT", "MEDICAL_HISTORY", "MEDICATION", "EXAMINATION", "DIAGNOSIS"]

for label in labels:
  nlp.get_pipe("ner").add_label(label)

In [ ]:
TRAIN_DATA = [
    ("Patient: John Doe Age: 45 Chief Complaint: Severe headache and dizziness", {
        "entities": [(8, 16, "PATIENT"), (22, 24, "AGE"), (41, 69, "CHIEF_COMPLAINT")]
    }),
    ("Medical Histroy: Hypertension, Type 2 Diabetes. ", {
        "entities": [(16, 29, "MEDICAL_HISTORY"), (31, 46, "MEDICAN_HUSTORY")]
    }),
    ("Current Medications: Metformin, Lisinporil.", {
        "entities": [(20, 29, "MEDICATION"), (31, 41, "MEDICATION")]
    }),
    ("Physical Examination: BP 150/90 mmHG, HR 88 bpm.", {
        "entities": [(20, 29, "EXAMINATION"), (31, 37, "EXAMINATION")]
    }),
    ("Diagnosis: Possible migraine, needs further evaluation.", {
        "entities": [(10, 27, "DIAGNOSIS")]
    })
]

In [ ]:
optimizer = nlp.create_optimizer()
for text, annotations in TRAIN_DATA:
  doc = nlp.make_doc(text)
  example = Example.from_dict(doc, annotations)
  nlp.update([example], sgd=optimizer)

summarizer = pipeline("summarization")

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Patient: John Doe Age: 45 Chief Complaint: Severe ..." with entities "[(8, 16, 'PATIENT'), (22, 24, 'AGE'), (41, 69, 'CH...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Medical Histroy: Hypertension, Type 2 Diabetes. " with entities "[(16, 29, 'MEDICAL_HISTORY'), (31, 46, 'MEDICAN_HU...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be align

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
clinical_note = """
Patient: John Doe, Age: 45, Chief Complaint: Severe headache and dizziness
Medical History: Hypertension, Type 2 Diabetes.
Current Medications: Metformin, Lisinporil.
Physical Examination: BP 150/90 mmHG, HR 88 bpm.
Diagnosis: Possible migraine, needs further evaluation.
"""

doc = nlp(clinical_note)
for ent in doc.ents:
  print(ent.text, ent.label_)

summary = summarizer(clinical_note, max_length = 50, min_length = 25, do_sample = False)
print("Summary:", summary[0]['summary_text'])

John Doe PERSON
45 CARDINAL
Type 2 Diabetes PERSON
150/90 CARDINAL
Summary:  Patient: John Doe, 45, Chief Complaint: Severe headache and dizziness . Current Medications: Metformin, Lisinporil, Metinol and Lisinol . Current BP 150/90 mmHG
